In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vehicles = pd.read_csv('/content/drive/MyDrive/vehicles_imputed.csv')

SyntaxError: ignored

In [ ]:
vehicles

,index,price,year,model,condition,cylinders,fuel,odometer,title_status,transmission,drive_n,type,paint_color
0,0,33590,2014.0,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,4wd,pickup,white
1,1,22590,2010.0,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,4wd,pickup,blue
2,2,39590,2020.0,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,4wd,pickup,red
3,3,30990,2017.0,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,4wd,pickup,red
4,4,15000,2013.0,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,truck,black
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256661,368780,41999,2015.0,2015,good,4 cylinders,gas,126000.0,clean,automatic,4wd,truck,white
256662,368786,6977,2011.0,civic,fair,4 cylinders,gas,112396.0,clean,automatic,fwd,sedan,white
256663,368787,4977,2005.0,altima,excellent,4 cylinders,gas,154917.0,clean,automatic,fwd,sedan,red
256664,368806,30990,2018.0,glc 300 sport,good,4 cylinders,gas,15080.0,clean,automatic,rwd,other,white


### Target - Predictor

In [ ]:
y = vehicles['price']
X = vehicles.drop(['price'], axis=1)

### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
X_cat = X.select_dtypes(include=['object'])
X_num = X.select_dtypes(include=['float64'])

In [ ]:
X_cat_col = X_cat.columns
for i in X_cat_col:
    X[i] = le.fit_transform(X[i])

In [ ]:
X.head()

,index,year,model,condition,cylinders,fuel,odometer,title_status,transmission,drive_n,type,paint_color
0,0,2014.0,21016,2,6,2,57923.0,0,2,0,8,11
1,1,2010.0,21321,2,6,2,71229.0,0,2,0,8,1
2,2,2020.0,21344,2,6,2,19160.0,0,2,0,8,9
3,3,2017.0,23932,2,6,2,41124.0,0,2,0,8,9
4,4,2013.0,12652,0,5,2,128000.0,0,0,2,10,0


In [ ]:
X.drop(['index'], inplace=True, axis=1)

### Train - Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=29)

## 3. Model 

In [ ]:
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model

### Evaluation

In [ ]:
def evaluation(y, pred):
    mae = mean_absolute_error(y, pred)
    rmse = np.sqrt(mean_squared_error(y, pred))
    r_squared = r2_score(y, pred)
    return mae, rmse, r_squared

In [ ]:
X.isnull().sum()

year            0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive_n         0
type            0
paint_color     0
dtype: int64

### 1) Random Forest

In [ ]:
parameter = [
    {'n_estimators':[100,200,300,500], 'max_features':[6,8,10,15]},
]

rf_reg = RandomForestRegressor()
grid_search = GridSearchCV(rf_reg, parameter,
                          cv=5, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
rf_final = grid_search.best_estimator_

In [ ]:
y_pred_rf = rf_final.predict(X_test)

In [ ]:
print(evaluation(y_test, y_pred_rf))

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
print(f'CV Score : {cross_val_score(estimator=rf_final, X=X_train , y=y_train, cv=10)}')


### 2) LightGBM

In [ ]:
! pip install lightgbm

In [ ]:
import lightgbm as lgb

In [ ]:
train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_test, y_test)

In [ ]:
params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'learning_rate': 0.001,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'max_bin' : 5000,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'min_data_in_leaf':800,
        'metric': 'rmse'
}

lgb_reg = lgb.train(params, train_set = train_set, num_boost_round=10000,
                   early_stopping_rounds=8000,verbose_eval=500, valid_sets=valid_set)



In [ ]:
y_pred_lgb = lgb_reg.predict(X_test)
print(evaluation(y_test, y_pred_lgb))

In [ ]:
y_pred_lgb_train = lgb_reg.predict(X_train)
print(evaluation(y_train, y_pred_lgb_train))